# Tool Wear Detection: Batch Prediction

## Prerequisites

1. Configure payload. 
1. Configure parser.
1. Configure batch ingestion spec.

```bash
curl --location --request POST 'http://34.160.53.45:8080/api/v1/ingestions' \
--header 'Content-Type: application/json' \
--data-raw '{
    "source": "JSON",
    "filePattern": "tool_wear_predictions\/prediction-(.*)\/(prediction|explanation).results-(.*)"
}'
```

## Setup

### Project ID

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: mde-ml-dev-01


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


## Explore MDE BigQuery

https://cloud.google.com/bigquery/docs/visualize-jupyter#query_notebook_data_using_the_bigquery_magic_command

https://googleapis.dev/python/bigquery/latest/magics.html

In [4]:
%%bigquery
SELECT
    schema_name
FROM
    INFORMATION_SCHEMA.SCHEMATA;

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.81rows/s]


,schema_name
0,ml
1,sfp_data
2,mde_looker_pdt


In [5]:
%%bigquery
SELECT
    * 
FROM 
    sfp_data.INFORMATION_SCHEMA.TABLES;

Downloading: 100%|██████████| 6/6 [00:01<00:00,  5.12rows/s]


,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,mde-ml-dev-01,sfp_data,NumericDataSeries,BASE TABLE,YES,NO,2022-09-08 16:05:32.847000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.NumericDa...,NULL,NaT
1,mde-ml-dev-01,sfp_data,InsertErrors,BASE TABLE,YES,NO,2022-09-08 16:05:32.821000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.InsertErr...,NULL,NaT
2,mde-ml-dev-01,sfp_data,ComponentDataSeries,BASE TABLE,YES,NO,2022-09-08 16:05:32.758000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.Component...,NULL,NaT
3,mde-ml-dev-01,sfp_data,DiscreteDataSeries,BASE TABLE,YES,NO,2022-09-08 16:05:33.153000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.DiscreteD...,NULL,NaT
4,mde-ml-dev-01,sfp_data,ContinuousDataSeries,BASE TABLE,YES,NO,2022-09-08 16:05:33.362000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.Continuou...,NULL,NaT
5,mde-ml-dev-01,sfp_data,OperationsDashboard,BASE TABLE,YES,NO,2022-09-08 16:05:33.063000+00:00,None,None,None,NaT,CREATE TABLE `mde-ml-dev-01.sfp_data.Operation...,NULL,NaT


In [6]:
%%bigquery
SELECT
  * 
FROM 
  sfp_data.NumericDataSeries
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.21rows/s]


,tagName,edgeTagName,eventTimestamp,value,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id
0,cncmilling_s1_commandvelocity,cncmilling_s1_commandvelocity,2022-09-22 15:32:04.791000+00:00,53.30000000000000000000000000000000000000,"{""value"":53.3}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 15:32:08.025679+00:00,"[{'key': 'value', 'value': '53.3'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'numeric-...",5717523514489386,3747547b-2fed-4647-addf-1b28f005cb45
1,cncmilling_x1_commandvelocity,cncmilling_x1_commandvelocity,2022-09-22 15:32:09.793000+00:00,0.72100000000000000000000000000000000000,"{""value"":0.721}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 15:32:14.514487+00:00,"[{'key': 'value', 'value': '0.721'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'numeric-...",5717436458991731,e803c5d7-1adb-4cf3-a1a4-417059af72e3
2,cncmilling_m1_current_program_number,cncmilling_m1_current_program_number,2022-09-22 15:32:34.799000+00:00,1.00000000000000000000000000000000000000,"{""value"":1}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 15:32:39.209704+00:00,"[{'key': 'value', 'value': '1'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'numeric-...",5717534678920223,d1243e9b-89f4-4023-b4fc-e93c7b17c372
3,cncmilling_z1_outputcurrent,cncmilling_z1_outputcurrent,2022-09-22 15:32:49.803000+00:00,0E-38,"{""value"":0}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 15:32:53.448216+00:00,"[{'key': 'value', 'value': '0'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'numeric-...",5717589137721466,4dd1f81d-30f4-446e-bed3-7474dda0c5fa
4,cncmilling_y1_positiondiff,cncmilling_y1_positiondiff,2022-09-22 15:32:49.803000+00:00,0.10000000000000000000000000000000000000,"{""value"":0.1}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 15:32:53.448235+00:00,"[{'key': 'value', 'value': '0.1'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'numeric-...",5717560127400615,813d15dd-4fce-4a93-acd5-108bc8d85841


In [7]:
%%bigquery
SELECT
  * 
FROM 
  sfp_data.DiscreteDataSeries
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.91rows/s]


,tagName,edgeTagName,eventTimestamp,eventType,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id
0,cncmilling_tool_condition,cncmilling_tool_condition,2022-09-22 04:49:44.412000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 04:49:48.805497+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'complex-...",5713856911532384,2e0798f3-8da5-4e1d-bbbf-04b4b3776038
1,cncmilling_tool_condition,cncmilling_tool_condition,2022-09-22 04:49:49.413000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 04:49:54.138687+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'complex-...",5713852822770447,71a11c3f-e5ad-4ec2-95c2-5166a058d519
2,cncmilling_tool_condition,cncmilling_tool_condition,2022-09-22 04:49:54.413000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 04:49:57.600391+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'complex-...",5713758507228499,4f0c473b-68bb-46f0-a759-c5cdff8547fb
3,cncmilling_tool_condition,cncmilling_tool_condition,2022-09-22 04:49:59.416000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 04:50:04.808840+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'complex-...",5713856578197986,dbd08d63-8ce8-4992-889f-95c09c670db6
4,cncmilling_tool_condition,cncmilling_tool_condition,2022-09-22 04:50:04.417000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/2/local/cncmill/1"":{""material"":""wax""...",2022-09-22 04:50:09.405125+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/2/local/cncmill...,"[{'key': 'headers.payload', 'value': 'complex-...",5713824367534748,05fdd34a-ae52-443e-957f-5e5090eb39a4


In [8]:
%%bigquery
SELECT
  DISTINCT tagName 
FROM 
  sfp_data.NumericDataSeries
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
  AND STARTS_WITH(tagName, 'cncmilling_')
ORDER BY tagName;

Downloading: 100%|██████████| 63/63 [00:01<00:00, 58.32rows/s]


,tagName
0,cncmilling_clamp_pressure
1,cncmilling_feedrate
2,cncmilling_m1_current_feedrate
3,cncmilling_m1_current_program_number
4,cncmilling_m1_sequence_number
...,...
58,cncmilling_z1_dcbusvoltage
59,cncmilling_z1_outputcurrent
60,cncmilling_z1_outputvoltage
61,cncmilling_z1_positiondiff


In [9]:
%%bigquery
SELECT
  DISTINCT tagName 
FROM 
  sfp_data.DiscreteDataSeries
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
  AND STARTS_WITH(tagName, 'cncmilling_')
ORDER BY tagName;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.25s/rows]


,tagName
0,cncmilling_tool_condition


In [10]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS ml
  OPTIONS (
    description = 'Dataset for storing machine learning artifacts',
    location = 'us'
  );

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 986.66query/s] 


""


In [11]:
%%bigquery
CREATE OR REPLACE TABLE ml.tool_wear_tags (
  tagName STRING,
  featureName STRING
) OPTIONS (
    description = 'Mapping between tags and feature for tool wear classification'
);

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 863.91query/s] 


""


Initialize tags-to-feature mapping table.

In [12]:
%%bigquery
INSERT ml.tool_wear_tags (tagName, featureName)
VALUES 
('cncmilling_clamp_pressure', 'clamp_pressure')
, ('cncmilling_feedrate', 'feedrate')
, ('cncmilling_m1_current_feedrate', 'M1_CURRENT_FEEDRATE')
, ('cncmilling_m1_current_program_number', 'M1_CURRENT_PROGRAM_NUMBER')
, ('cncmilling_m1_sequence_number', 'M1_sequence_number')
, ('cncmilling_machining_process', 'Machining_Process')
, ('cncmilling_s1_accelerationdiff', 'S1_AccelerationDiff')
, ('cncmilling_s1_actualacceleration', 'S1_ActualAcceleration')
, ('cncmilling_s1_actualposition', 'S1_ActualPosition')
, ('cncmilling_s1_actualvelocity', 'S1_ActualVelocity')
, ('cncmilling_s1_commandacceleration', 'S1_CommandAcceleration')
, ('cncmilling_s1_commandposition', 'S1_CommandPosition')
, ('cncmilling_s1_commandvelocity', 'S1_CommandVelocity')
, ('cncmilling_s1_currentfeedback', 'S1_CurrentFeedback')
, ('cncmilling_s1_dcbusvoltage', 'S1_DCBusVoltage')
, ('cncmilling_s1_outputcurrent', 'S1_OutputCurrent')
, ('cncmilling_s1_outputpower', 'S1_OutputPower')
, ('cncmilling_s1_outputvoltage', 'S1_OutputVoltage')
, ('cncmilling_s1_positiondiff', 'S1_PositionDiff')
, ('cncmilling_s1_systeminertia', 'S1_SystemInertia')
, ('cncmilling_s1_velocitydiff', 'S1_VelocityDiff')
, ('cncmilling_x1_accelerationdiff', 'X1_AccelerationDiff')
, ('cncmilling_x1_actualacceleration', 'X1_ActualAcceleration')
, ('cncmilling_x1_actualposition', 'X1_ActualPosition')
, ('cncmilling_x1_actualvelocity', 'X1_ActualVelocity')
, ('cncmilling_x1_commandacceleration', 'X1_CommandAcceleration')
, ('cncmilling_x1_commandposition', 'X1_CommandPosition')
, ('cncmilling_x1_commandvelocity', 'X1_CommandVelocity')
, ('cncmilling_x1_currentfeedback', 'X1_CurrentFeedback')
, ('cncmilling_x1_dcbusvoltage', 'X1_DCBusVoltage')
, ('cncmilling_x1_outputcurrent', 'X1_OutputCurrent')
, ('cncmilling_x1_outputpower', 'X1_OutputPower')
, ('cncmilling_x1_outputvoltage', 'X1_OutputVoltage')
, ('cncmilling_x1_positiondiff', 'X1_PositionDiff')
, ('cncmilling_x1_velocitydiff', 'X1_VelocityDiff')
, ('cncmilling_y1_accelerationdiff', 'Y1_AccelerationDiff')
, ('cncmilling_y1_actualacceleration', 'Y1_ActualAcceleration')
, ('cncmilling_y1_actualposition', 'Y1_ActualPosition')
, ('cncmilling_y1_actualvelocity', 'Y1_ActualVelocity')
, ('cncmilling_y1_commandacceleration', 'Y1_CommandAcceleration')
, ('cncmilling_y1_commandposition', 'Y1_CommandPosition')
, ('cncmilling_y1_commandvelocity', 'Y1_CommandVelocity')
, ('cncmilling_y1_currentfeedback', 'Y1_CurrentFeedback')
, ('cncmilling_y1_dcbusvoltage', 'Y1_DCBusVoltage')
, ('cncmilling_y1_outputcurrent', 'Y1_OutputCurrent')
, ('cncmilling_y1_outputpower', 'Y1_OutputPower')
, ('cncmilling_y1_outputvoltage', 'Y1_OutputVoltage')
, ('cncmilling_y1_positiondiff', 'Y1_PositionDiff')
, ('cncmilling_y1_velocitydiff', 'Y1_VelocityDiff')
, ('cncmilling_z1_accelerationdiff', 'Z1_AccelerationDiff')
, ('cncmilling_z1_actualacceleration', 'Z1_ActualAcceleration')
, ('cncmilling_z1_actualposition', 'Z1_ActualPosition')
, ('cncmilling_z1_actualvelocity', 'Z1_ActualVelocity')
, ('cncmilling_z1_commandacceleration', 'Z1_CommandAcceleration')
, ('cncmilling_z1_commandposition', 'Z1_CommandPosition')
, ('cncmilling_z1_commandvelocity', 'Z1_CommandVelocity')
, ('cncmilling_z1_currentfeedback', 'Z1_CurrentFeedback')
, ('cncmilling_z1_dcbusvoltage', 'Z1_DCBusVoltage')
, ('cncmilling_z1_outputcurrent', 'Z1_OutputCurrent')
, ('cncmilling_z1_outputvoltage', 'Z1_OutputVoltage')
, ('cncmilling_z1_positiondiff', 'Z1_PositionDiff')
, ('cncmilling_z1_velocitydiff', 'Z1_VelocityDiff')
, ('cncmilling_tool_condition', 'tool_condition')
, ('cncmilling_material', 'material');

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1396.08query/s]                        


""


In [13]:
%%bigquery
SELECT
  *
FROM
  `ml.tool_wear_tags`
ORDER BY
  tagName 
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.43rows/s]


,tagName,featureName
0,cncmilling_clamp_pressure,clamp_pressure
1,cncmilling_feedrate,feedrate
2,cncmilling_m1_current_feedrate,M1_CURRENT_FEEDRATE
3,cncmilling_m1_current_program_number,M1_CURRENT_PROGRAM_NUMBER
4,cncmilling_m1_sequence_number,M1_sequence_number
5,cncmilling_machining_process,Machining_Process
6,cncmilling_material,material
7,cncmilling_s1_accelerationdiff,S1_AccelerationDiff
8,cncmilling_s1_actualacceleration,S1_ActualAcceleration
9,cncmilling_s1_actualposition,S1_ActualPosition


In [14]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = '''
CREATE OR REPLACE PROCEDURE ml.create_prediction_features_view(view_name STRING)
BEGIN
    DECLARE select_expr STRING;
    DECLARE pivot_expr STRING;
    DECLARE query STRING;
    DECLARE last_prediction_timestamp DEFAULT (
      SELECT
        IFNULL(
          MAX(eventTimestamp),
          TIMESTAMP_SECONDS(0)
        ) AS last_prediction_timestamp
      FROM
        `sfp_data.DiscreteDataSeries`
      WHERE
        DATE(eventTimestamp) >= DATE_FROM_UNIX_DATE(0)
        AND tagName = 'tool_wear_predictions'
    );
    
    -- Dynamically create select expressions
    SET select_expr = (
      WITH SelectExpr AS (
        SELECT 
          CASE
            WHEN EXISTS (SELECT DISTINCT tagName FROM `sfp_data.DiscreteDataSeries` d WHERE d.eventTimestamp > last_prediction_timestamp AND d.tagName = t.tagName)
              THEN FORMAT(', JSON_EXTRACT_SCALAR(payload_%s, "$") AS %s', t.tagName, t.featureName)
            WHEN EXISTS (SELECT DISTINCT tagName FROM `sfp_data.NumericDataSeries` n WHERE n.eventTimestamp > last_prediction_timestamp AND n.tagName = t.tagName)
              THEN FORMAT(', JSON_EXTRACT_SCALAR(payload_%s, "$.value") AS %s', t.tagName, t.featureName)
            ELSE ''
          END AS expr
        FROM
          `ml.tool_wear_tags` t
      )
      SELECT 
        STRING_AGG(SelectExpr.expr, ' ')
      FROM 
        SelectExpr
    );
    
    -- Dynamically create pivot expression
    SET pivot_expr = (
      SELECT
        STRING_AGG(DISTINCT CONCAT("'", tagName, "'"))
      FROM
        `ml.tool_wear_tags`
    );
    
    -- Format query using dynamic select and pivot expressions
    SET query = (
      SELECT FORMAT(r"""
        CREATE OR REPLACE VIEW %s AS
        WITH metadata AS (
            SELECT
              d.eventTimestamp
              , CASE
                  WHEN (REGEXP_CONTAINS(mkv.schemaIdentifier, r'^(cncmill/\d+/local/cncmill/\d+)$') AND mkv.key = 'material')
                    THEN 'cncmilling_material'
                  ELSE NULL
                END AS tagName
              , TO_JSON_STRING(mkv.value) AS payload
            FROM
              `sfp_data.DiscreteDataSeries` d, d.metadataKV mkv
            WHERE
              TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) > "%t"
              AND d.tagName = 'cncmilling_tool_condition'
              AND ARRAY_LENGTH(metadataKV) != 0
        )
        SELECT 
          eventTimestamp
          %s
          , JSON_EXTRACT_SCALAR(payload_cncmilling_material, "$") AS material
        FROM 
        (
          SELECT
            TIMESTAMP_TRUNC(n.eventTimestamp, SECOND) AS eventTimestamp
            , n.tagName
            , n.payload
          FROM
            `sfp_data.NumericDataSeries` n
          INNER JOIN
            `ml.tool_wear_tags` ntag
          ON
            n.tagName = ntag.tagName
          WHERE
            TIMESTAMP_TRUNC(n.eventTimestamp, SECOND) > "%t"
          
          UNION ALL
          
          SELECT
            TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) AS eventTimestamp
            , d.tagName
            , d.payload
          FROM
            `sfp_data.DiscreteDataSeries` d
          INNER JOIN
            `ml.tool_wear_tags` dtag
          ON
            d.tagName = dtag.tagName
          WHERE
            TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) > "%t"
          
          UNION ALL
          
          SELECT
            TIMESTAMP_TRUNC(metadata.eventTimestamp, SECOND) AS eventTimestamp
            , metadata.tagName
            , metadata.payload
          FROM
            metadata
          INNER JOIN
            `ml.tool_wear_tags` mtag
          ON
            metadata.tagName = mtag.tagName
        )
        PIVOT
        (
          ANY_VALUE(payload) as payload
          FOR tagName
          IN (
            %s
          )
        )
    """,
        view_name,
        last_prediction_timestamp, 
        select_expr, 
        last_prediction_timestamp, 
        last_prediction_timestamp, 
        pivot_expr)
    );

    -- Execute query
    EXECUTE IMMEDIATE query;
END
'''

results = bq_client.query(query)

results.to_dataframe()

""


In [15]:
from datetime import datetime

current_time_string = datetime.now().strftime('%Y%m%d_%H%M%S')
view_name = f'ml.prediction_features_{current_time_string}'
bq_params = {'view_name': view_name}

view_name

'ml.prediction_features_20220922_154959'

In [16]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''CALL ml.create_prediction_features_view('{view_name}');'''

results = bq_client.query(query)

results.to_dataframe()

""


In [17]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  * 
FROM 
  ml.INFORMATION_SCHEMA.VIEWS
WHERE
  table_name = '{view_name.split('.')[1]}';
'''

results = bq_client.query(query)
df = results.to_dataframe()

print(df['view_definition'].values[0])

WITH metadata AS (
            SELECT
              d.eventTimestamp
              , CASE
                  WHEN (REGEXP_CONTAINS(mkv.schemaIdentifier, r'^(cncmill/\d+/local/cncmill/\d+)$') AND mkv.key = 'material')
                    THEN 'cncmilling_material'
                  ELSE NULL
                END AS tagName
              , TO_JSON_STRING(mkv.value) AS payload
            FROM
              `sfp_data.DiscreteDataSeries` d, d.metadataKV mkv
            WHERE
              TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) > "2022-09-21 13:12:24+00"
              AND d.tagName = 'cncmilling_tool_condition'
              AND ARRAY_LENGTH(metadataKV) != 0
        )
        SELECT 
          eventTimestamp
          , JSON_EXTRACT_SCALAR(payload_cncmilling_x1_commandacceleration, "$.value") AS X1_CommandAcceleration , JSON_EXTRACT_SCALAR(payload_cncmilling_s1_velocitydiff, "$.value") AS S1_VelocityDiff  , JSON_EXTRACT_SCALAR(payload_cncmilling_z1_velocitydiff, "$.value") AS Z1_VelocityDi

In [18]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT *
FROM `{view_name}`
ORDER BY eventTimestamp DESC
LIMIT 10;
'''

results = bq_client.query(query)

results.to_dataframe()

,eventTimestamp,X1_CommandAcceleration,S1_VelocityDiff,Z1_VelocityDiff,X1_CommandVelocity,Z1_ActualPosition,Machining_Process,X1_ActualVelocity,S1_AccelerationDiff,Z1_CommandAcceleration,...,Z1_CurrentFeedback,Y1_CommandVelocity,X1_ActualAcceleration,Y1_DCBusVoltage,S1_OutputCurrent,S1_PositionDiff,Y1_OutputPower,feedrate,S1_CommandVelocity,material
0,2022-09-22 15:50:00+00:00,0,0.001,0.025,0,119,8,0,0.5,0,...,0,0,0,2.68E-19,331,0,2.68E-7,20,0,wax
1,2022-09-22 15:49:55+00:00,0,0.001,0,0,119,8,0,0.75,0,...,0,0,0,2.69E-19,323,0,-5.38E-7,20,0,wax
2,2022-09-22 15:49:50+00:00,0,0.001,0.025,0,119,8,0,0.5,0,...,0,0,0,2.68E-19,331,0,2.68E-7,20,0,wax
3,2022-09-22 15:49:45+00:00,0,0,0,0,119,8,0,0.25,0,...,0,0,0,2.69E-19,323,0,6.7E-7,20,0,wax
4,2022-09-22 15:49:40+00:00,0,0,0,0,119,8,0,0.25,0,...,0,0,0,2.68E-19,331,0,5.12E-7,20,0,wax
5,2022-09-22 15:49:35+00:00,0,0,0,0,119,8,0,0.25,0,...,0,0,0,2.68E-19,331,0,5.12E-7,20,0,wax
6,2022-09-22 15:49:30+00:00,30.1,0.1,0,-19.5,28.5,8,-19.3,56.8,0,...,0,4.24,25,0.105,322,0,7.43E-4,20,53.3,wax
7,2022-09-22 15:49:25+00:00,0,0.1,0.05,-20,28.5,8,-20.2,47.6,0,...,0,0,-37.5,0.0208,322,0,-1.02E-6,20,53.3,wax
8,2022-09-22 15:49:20+00:00,0,0.2,0,-20,28.5,8,-20,51,0,...,0,0,-6.25,0.0184,322,0,-3.46E-7,20,53.3,wax
9,2022-09-22 15:49:15+00:00,0,0,0.05,-20,28.5,8,-19.8,19.1,0,...,0,0,56.3,0.0186,321,0,7.57E-7,20,53.3,wax


## Vertex AI Batch Prediction

In [19]:
from datetime import datetime

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import Model
from google.cloud.aiplatform_v1beta1.services.job_service import JobServiceClient
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

vertex_ai.init(project=PROJECT_ID)

### Global variables

In [20]:
LOCATION = 'us-central1'
API_ENDPOINT = f'{LOCATION}-aiplatform.googleapis.com'
CLIENT_OPTIONS = {"api_endpoint": API_ENDPOINT}
PARENT = f"projects/{PROJECT_ID}/locations/{LOCATION}"

BQ_ML_DATASET = 'ml'
BQ_PREDICTION_INPUT_TABLE = f'{BQ_ML_DATASET}.unpredicted_data'
NAME_PREFIX = 'tool_wear'
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [21]:
MODEL_NAME = "[your-model-name]" # Format: 'projects/{project}/locations/{location}/models/{model_id}'

if MODEL_NAME == "" or MODEL_NAME is None or MODEL_NAME == "[your-model-name]":
    # Get latest updated model with prefix
    models = Model.list(
        order_by='updateTime desc',
    )

    MODEL_NAME = None
    for m in models:
        if m.display_name.startswith(NAME_PREFIX):
            MODEL_NAME = m.resource_name
            print(f'Vertex AI model found: {m.display_name}')
            break

    if MODEL_NAME is None:
        print(f'Vertex AI model with prefix "{NAME_PREFIX}" not found.'
              f'Please search using a new prefix or locate the model resource name manually.')

Vertex AI model found: tool_wear_v3_split3


In [22]:
TRAINING_DATASET_NAME = "[your-dataset-name]" # Format: 'projects/{project}/locations/{location}/datasets/{dataset_id}'

if TRAINING_DATASET_NAME == "" or TRAINING_DATASET_NAME is None or TRAINING_DATASET_NAME == "[your-dataset-name]":
    # Get latest updated dataset with prefix
    client = vertex_ai.gapic.DatasetServiceClient(client_options=CLIENT_OPTIONS)
    datasets = client.list_datasets(request={
        'parent': PARENT,
        'order_by': 'update_time desc',
    })
    
    TRAINING_DATASET_NAME = None
    for d in datasets:
        if d.display_name.startswith(NAME_PREFIX):
            TRAINING_DATASET_NAME = d.name
            print(f'Vertex AI dataset found: {d.display_name}')
            break
    
    if TRAINING_DATASET_NAME is None:
        print(f'Vertex AI dataset with prefix "{NAME_PREFIX}" not found.'
              f'Please search using a new prefix or locate the dataset resource name manually.')

Vertex AI dataset found: tool_wear_v3_split3


### Create Batch Prediction Job

Reference: https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#aiplatform_create_batch_prediction_job_bigquery_sample-python

In [23]:
DISPLAY_NAME = f'tool_wear_{TIMESTAMP}'
model_parameters_dict = {}
MODEL_PARAMETERS = json_format.ParseDict(model_parameters_dict, Value())
INSTANCE_FORMAT = 'bigquery' # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models#Model.FIELDS.supported_input_storage_formats
BQ_SOURCE_URI = f'bq://{PROJECT_ID}.{view_name}' # f'bq://{PROJECT_ID}.{BQ_PREDICTION_INPUT_TABLE}' # Format: bq://projectId.bqDatasetId.bqTableId
PREDICTIONS_FORMAT = 'jsonl' # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models#Model.FIELDS.supported_output_storage_formats
GCS_OUTPUT_URI_PREFIX = f'gs://{PROJECT_ID}-batch-ingestion/tool_wear_predictions/' # Format: gs://bucket_name/folder_name
MACHINE_TYPE = 'n1-standard-2'
MIN_NODES = 1
MAX_NODES = 3

# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = JobServiceClient(client_options=CLIENT_OPTIONS)

batch_prediction_job = {
    "display_name": DISPLAY_NAME,
    "model": MODEL_NAME,
    "model_parameters": MODEL_PARAMETERS,
    "input_config": {
        "instances_format": INSTANCE_FORMAT,
        "bigquery_source": {"input_uri": BQ_SOURCE_URI},
    },
    "output_config": {
        "predictions_format": PREDICTIONS_FORMAT,
        "gcs_destination": {"output_uri_prefix": GCS_OUTPUT_URI_PREFIX},
    },
    "dedicated_resources": {
        'machine_spec': {
            "machine_type": MACHINE_TYPE,
        },
        'starting_replica_count': MIN_NODES,
        'max_replica_count': MAX_NODES,
    },
    "generate_explanation": True,
    "explanation_spec": {
        "parameters": {
            "sampled_shapley_attribution": {
                "path_count": 3
            }
        }
    },
    # 'model_monitoring_config': {
    #     'alert_config': {
    #         'enable_logging': True
    #     },
    #     'objective_configs': [
    #         {
    #             'training_dataset': {
    #                 'dataset': TRAINING_DATASET_NAME
    #             },
    #         }
    #     ],
    # }
}


response = client.create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

print("response:", response.name)

response: projects/944080718770/locations/us-central1/batchPredictionJobs/8367460233869524992


In [24]:
# TODO: configure batch prediction explanation spec
# https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/36455b8125802459f3a40752fbda0e4c9407c854/notebooks/community/sdk/SDK_Explainable_AI_Custom_Tabular.ipynb
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.types.BatchPredictionJob
# TODO: Explore prediction data

## Explore Predictions Data

You may have to wait for the batch prediction job to finish. 

In [25]:
%%bigquery
SELECT
  *
FROM
  `mde-ml-dev-01.sfp_data.DiscreteDataSeries`
WHERE
  DATE(eventTimestamp) > DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY)
  AND tagName = 'tool_wear_predictions'
ORDER BY
  eventTimestamp DESC
LIMIT
  10;

Query complete after 0.00s: 100%|██████████| 2/2 [00:00<00:00, 823.06query/s]                         
Downloading: 0rows [00:00, ?rows/s]


,tagName,edgeTagName,eventTimestamp,eventType,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id


## Clean Up

In [27]:
from google.cloud import bigquery

bq_client = bigquery.Client()
query = f'''DROP VIEW IF EXISTS {view_name};'''
results = bq_client.query(query)

results.to_dataframe()

""
